In [3]:
#Author: Yohannes 
#Importing packages 
import requests
import pandas as pd 
import datetime 
import base64
from urllib.parse import urlencode
import json
import psycopg2 as ps 

In [ ]:
#Getting client_id and client_secret from spotify
#Client credentials 
 
client_id = "xxxxxxxxx"
client_secret = "xxxxxxxxx"

#Encoding client_id and client_secret using base64
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())
client_creds_b64

In [ ]:
#Authorization to grant spotify access permission to extract data 

token_url = "https://accounts.spotify.com/api/token"
method = "POST"

token_data = {
          "grant_type": "client_credentials"
}

token_headers = {
        "Authorization": f"Basic {client_creds_b64.decode()}"
}
token_headers

In [5]:
req = requests.post(token_url, data = token_data, headers = token_headers)
token_response_data = req.json()
req.status_code

200

In [ ]:
token_response_data = req.json()
access_token = token_response_data['access_token']
expires_in = token_response_data['expires_in'] #the time is in seconds
token_type = token_response_data['token_type']
token_response_data

In [ ]:
token_headers = {
    "Authorization": f"Bearer {access_token}"
}

endpoint = "https://api.spotify.com/v1/search"
data = urlencode({
    "q": "R&B",
    "type": "artist"
})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)

res = requests.get(lookup_url, headers=token_headers)
res = res.json()
res

In [ ]:
artists = res['artists']['items']
df = pd.json_normalize(artists)
df.info()

In [ ]:
# drop columns: followers href, images, href, uri 
df = df.drop(['href', 'images', 'uri', 'followers.href', 'external_urls.spotify'], axis=1,)
df.head()

In [ ]:
#Popularity
df.groupby(['name'])['popularity'].sum().sort_values(ascending=False)

In [ ]:
#Number of followers
df.groupby(['name'])['followers.total'].sum().sort_values(ascending=False)

In [ ]:
# Connecting to Rds database 
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname,
                          user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print("connected")

    return conn

In [ ]:
host_name = "xxxxx"
dbname = 'xxxxx'
port = 'xxxx'
username = 'postgres'
password = 'xxxxxxxxx'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)